Here is a hodgepodge of Astropy capabilities...

# Astropy

## Photometry with Photutils

What is photometry?  It is the measurement of light from a source, either represented as flux or magnitude. There are several ways to compute the flux from a source. In this tutorial we will go over aperture photometry, where you draw a circular region around the source and measure the flux inside.

[Photoutils](https://photutils.readthedocs.io) is a useful astropy package for performing photometry.  This includes but not limited to aperture photometry, PSF photometry, star finding, source isophotes, background subtraction, etc...

Large parts of this tutorial are borrowed from [Getting Started with Photutils](https://photutils.readthedocs.io/en/stable/getting_started.html) documentation.

Here we load in a star field image provided with Photutils.  We are interested in a small subsection of the image in which we want to do photometry.

In [ ]:
import numpy as np
from photutils import datasets

hdu = datasets.load_star_image()  
image = hdu.data[500:700, 500:700].astype(float)
head = hdu.header

Assuming your background is uniform (and it most certianly is not), here is a rough background subtraction. 

This may not work for all astronomical fields (e.g. clusters, low surface brightness features, etc). There are more sophisticated ways to deal with the background subtraction, see the [Background Estimation](https://photutils.readthedocs.io/en/stable/background.html) documentation.

In [ ]:
image -= np.median(image) 

We use [DAOStarFinder](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html) to find all of the objects in the image. 

The FWHM (Full Width Half Maximum) is a way of measureing the size of the source. Often for unresolved sources, like stars, the FWHM should be matched the resolution of the telescope (or the seeing).  

The threshold is the value above the background in which the star finding algorithm will find sources. Often this should be done in a statistical sense in which a real detection is n-sigma above the background (or n standard deviations). We can measure the standard deviation of background. 

Statistics aside:
Assuming a Gaussian distribution for the noise, 1-sigma contains 68% of the distribution, 2-sigma contains 95% of the distribution and 3-sigma contains 99.7% of the distribution. Often 5-sigma is the golden standard in physics and astronomy for a "real" detection as it is contains 99.99994% background distribution.  

Here we use 3-sigma (3 standard devations) above the background for a "real" detection. 

In [ ]:
from photutils import DAOStarFinder
from astropy.stats import mad_std

bkg_sigma = mad_std(image)

daofind = DAOStarFinder(fwhm=4., threshold=3.*bkg_sigma)  
sources = daofind(image)

for col in sources.colnames: sources[col].info.format = '%.8g'  # for consistent table output
print(sources[:10])
print(sources.info)

In [ ]:
from photutils import aperture_photometry, CircularAperture

positions = np.transpose((sources['xcentroid'], sources['ycentroid']))  
apertures = CircularAperture(positions, r=4.)  
phot_table = aperture_photometry(image, apertures)

for col in phot_table.colnames: phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table[:10])
print(phot_table.info)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image, cmap='gray_r', origin='lower')
apertures.plot(color='blue', lw=1.5, alpha=0.5)

In [ ]:
print(head.keys())

## Astroquery

Astroquery is a Astropy package for querying astronomical databases.  There many, many, many  catalogs and image services.  Astroquery can be used to find catalogs and images.  In this tutorial, we will show a few different databases and and ways to search for data.

### NED query

NED is the [NASA/IPAC Extragalacitc Database](https://ned.ipac.caltech.edu/).  It can be useful for finding measurements, properties and papers about
your favorite galaxy (or galaxies).  Like many databases you can search for galaxies via their name or coordinates.

First we need to import the modules that we will use for this tutorial.

In [ ]:
from astroquery.ned import Ned
from astropy import coordinates
import astropy.units as u

In [ ]:
result_table = Ned.query_object("m82")
print(result_table)

Print the columns returned from the search.

In [ ]:
print(result_table.keys())

Print the redshift of the source

In [ ]:
print(result_table['Redshift'])

Lets now search for objects around M82.

In [ ]:
result_table = Ned.query_region("m82", radius=1 * u.arcmin)
print(result_table)

### GAIA query

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
coord = coordinates.SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
radius = u.Quantity(1.0, u.arcmin)
j = Gaia.cone_search_async(coord, radius)
r = j.get_results()

r.pprint()
print(r.keys())

### NOIRLab query...

### IRSA query of 2MASS

IRSA is the [NASA/IPAC Infrared Science Archive](https://irsa.ipac.caltech.edu).  IRSA typically has a infrared mission focus, such as 2MASS, Herschel, Planck, Spitzer, WISE, etc...

In [ ]:
from astroquery.irsa import Irsa

In [ ]:
Irsa.list_catalogs()

In [ ]:
table = Irsa.query_region("m82", catalog="fp_psc", spatial="Cone",radius=10 * u.arcmin)
#print(table)
#print(table.keys())

### SDSS query

In [ ]:
from astroquery.sdss import SDSS

Coordinates of your favorite object, in this case M82.

In [ ]:
ra, dec = 148.969687, 69.679383

In [ ]:
co = coordinates.SkyCoord(ra=ra, dec=dec,unit=(u.deg, u.deg), frame='fk5')

In [ ]:
xid = SDSS.query_region(co, radius=10 * u.arcmin)
# print the first 10 entries
print(xid[:10])
print(xid.keys())

In [ ]:
print(xid['ra','dec'][:10]) # print the first 10 entries

### Skyview

[Skyview](https://skyview.gsfc.nasa.gov) is a virtual observatory that is connected to several surveys and imaging data sets spanning multiple wavelengths. It can be a handy way to get quick access to multi-wavelength imaging data for your favorite object.

In [ ]:
from astroquery.skyview import SkyView

SkyView.list_surveys()

In [ ]:
#pflist = SkyView.get_images(position='M82', survey=['2MASS-K'],radius=10 * u.arcmin)
pflist = SkyView.get_images(position='M82', survey=['SDSSr'],radius=10 * u.arcmin)

In [ ]:
ext = 0
pf = pflist[0] # first element of the list, might need a loop if multiple images
m82_image = pf[ext].data

Plot image

In [ ]:
ax = plt.subplot()
ax.imshow(m82_image, cmap='gray_r', origin='lower', vmin=-10, vmax=20)
ax.set_xlabel('X (pixels)')
ax.set_ylabel('Y (pixels)')

Plot the in RA and Declination coordinates space, instead of pixels.

In [ ]:
from astropy.wcs import WCS

In [ ]:
wcs = WCS(pf[ext].header)

ax = plt.subplot(projection=wcs)
ax.imshow(m82_image, cmap='gray_r', origin='lower', vmin=-10, vmax=20)
#ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension (J2000)')
ax.set_ylabel('Declination (J2000)')

#ax.scatter(xid['ra'],xid['dec'],marker="o",s=50,transform=ax.get_transform('fk5'),edgecolor='b', facecolor='none')

## More...

## Resources

http://learn.astropy.org/tutorials.html - list of all the Astropy tutorials

https://astroquery.readthedocs.io - astroquery documentation

https://photutils.readthedocs.io - photutils documentation